In [1]:
%run setup.py

In [2]:
data = pd.read_excel('../sample/samples.xlsx')

idx = ~np.isfinite(data['Dnu'])
data.loc[idx, 'Dnu'] = data.loc[idx, 'Dnu_guess']

idx = np.isfinite(data['ifmodelling']) & np.isfinite(data['Dnu']) & (np.isfinite(data['lum_J'])) & \
        (data['[M/H]'] > -0.7) & (~np.isin(data['names'], ['binary', 'Procyon/A', 'Procyon/B']))

data = data[idx].reset_index(drop=True)

In [4]:
# data[['KIC', 'lum_J', 'Teff', 'Dnu', 'numax', '[M/H]']]

lum, e_lum, r_lum = '$L$', r'$\sigma_L$', 'Ref($L$)'
Teff, e_Teff, r_Teff = r'$T_{\rm eff}$', r'$\sigma_{T_{\rm eff}}$', r'Ref($T_{\rm eff}$)'
mh, e_mh, r_mh = r'${\rm [M/H]}$', r'$\sigma_{\rm [M/H]}$', r'Ref(${\rm [M/H]}$)'

data['Star'] = 'KIC'+data['KIC'].astype('str')
data.loc[data['KIC']==1, 'Star'] = r'$\mu$ Her'
data.loc[data['KIC']==0, 'Star'] = '$\rm Sun$'

# Luminosity
data[lum] = data['lum_J'].round(4)
data[e_lum] = data['e_lum_J'].round(4)
data[r_lum] = '0'
data.loc[data['KIC']==1, r_lum] = '2'
data.loc[data['KIC']==0, r_lum] = '---'


# Teff
data[Teff] = data['Teff'].round(0)
data[e_Teff] = data['e_Teff'].round(0)
data.loc[np.abs(data[Teff]-data['Teff_phot'])<0.01, r_Teff] = '1'
data.loc[data['KIC']==1, r_Teff] = '3'
data.loc[data['KIC']==0, r_Teff] = '---'

# mh
data[mh] = data['[M/H]'].round(3)
data[e_mh] = 0.05
data.loc[data['KIC']==1, r_mh] = '3'
data.loc[data['KIC']==0, r_mh] = '---'
data.loc[np.abs(data[mh]-(data['[Fe/H]_F18']+0.01))<0.005, r_mh] = '4'
data.loc[np.abs(data[mh]-(data['[Fe/H]_L17']+0.03))<0.005, r_mh] = '5'
data.loc[np.abs(data[mh]-(data['[Fe/H]_keck']-0.09))<0.005, r_mh] = '0'
data.loc[np.abs(data[mh]-(data['[M/H]_B15']+0.02))<0.005, r_mh] = '7'
data.loc[np.abs(data[mh]-data['[M/H]_apo'])<0.0001, r_mh] = '6'
data.loc[np.isin(data['names'], ['ngc6819', 'ngc6791']), r_mh] = '6'
# data.loc[data[mh]==data['[Fe/H]_'], r_mh] = '6'

# Dnu
# data[r'$\Delta\nu$'] = data['Dnu']
# data[r'$\sigma_{\Delta\nu}$'] = data['e_Dnu']
# data['Ref_Dnu'] = 

# data['numax'] = r'$\nu_{\rm max}$'
# data['e_numax'] = 
# data['Ref_numax'] = 

refs_dict = {
'0': '(This work)',
'1': '\citep{casagrande++2021-irfm-gaia}',
'2': '\citep{grundahl++2017-mu-her}', 
'3': '\citep{jofre++2015-star-params}',
'4': '\citep{furlan++2018-kepler}',
'5': '\citep{lund++2017-legacy-kepler-1}',
'6': '\citep{ahunmada++2020-apogee-dr16}',
'7': '\citep{buchhave++2015-feh}',
}


In [5]:
data[r_mh].value_counts()

6      1190
4        36
0        21
7         6
5         5
3         1
---       1
Name: Ref(${\rm [M/H]}$), dtype: int64

In [6]:
# idx = data[r_mh].isna() | data[r_lum].isna() | data[r_Teff].isna()
# data.index[data['Star']=='Sun']=0
data.loc[:, ['Star', #'stage', 'names',
     lum, e_lum, r_lum, 
     Teff, e_Teff, r_Teff,
     mh, e_mh, r_mh]].sort_values('Star').reset_index(drop=True).to_csv('Tab.1_stellar_params_full.txt', index=False)#.head(10)

# full table first two rows
#Star,L,e_L,ref_L,Teff,e_Teff,ref_Teff,[M/H],e_[M/H],ref_[M/H]
# Sun,1.0,0.02,---,5777.0,100.0,---,0.0,0.05,---

In [7]:
data = data.sort_values('Star').iloc[0:9]

In [8]:
def my_write_latex_table(listofcol, listofcolerr, listofcolname, listofcolformat, listofcolerrformat, filename, expandcolumns=True, scaption='', slabel='', snote='', sfootnote='',align=None):
    # version 2 - modified in 20/Dec/2018 - added footnote

    # test if input lists are of equal length
    a, b, c, d, e = [len(obj) for obj in [listofcol, listofcolerr, listofcolname, listofcolformat, listofcolerrformat]]
    assert a == b == c == d == e

    # configure footnote
    if sfootnote == '':
        footnote = ''
    else:
        footnote = '\n \\begin{tablenotes}  \n \\item \\emph{Note}: ' + sfootnote + '\n \\end{tablenotes}'

    # create a pandas table

    listofdata = [[listofcolformat[tcol].format(listofcol[tcol][trow]) + listofcolerrformat[tcol].format(listofcolerr[tcol][trow])  for trow in range(len(listofcol[0]))] 
        for tcol in range(len(listofcol))]

    table = pd.DataFrame(np.array(listofdata).T, columns = listofcolname)

    f = open(filename,'w')

    stable = 'table*' if expandcolumns else 'table'
    f.write('\\begin{'+stable+'} \n')
#     f.write('\tiny \n')
    f.write('\caption{'+scaption+' \n')
    f.write('\label{'+slabel+'} \\\\ \n')
    f.write(snote+'} \n')


    ncol = table.shape[1]
    if align is None: align = 'l'+'c'*ncol
    s = table.to_latex(index=False, float_format='%.2f', escape=False, column_format=''+align) #@{\extracolsep{\\fill}}
    width = 'textwidth' if expandcolumns else 'columnwidth'
    s = s.replace('\\begin{tabular}', '\\begin{tabular*}{1.01\\'+width+'}')
    s = s.replace('\\end{tabular}', '\\end{tabular*}'+footnote)

    f.write(s)

    f.write('\\end{'+stable+'} \n')
    f.close()

In [183]:
listofcol1 = [ data['Star'].to_numpy(),
               data[lum].round(2).astype(str).to_numpy(),
               data[e_lum].round(2).astype(str).to_numpy(),
               data[r_lum].astype(str).to_numpy(),
               data[Teff].astype(int).astype(str).to_numpy(),
               data[e_Teff].astype(int).astype(str).to_numpy(),
               data[r_Teff].astype(str).to_numpy(),
               data[mh].round(2).astype(str).to_numpy(),
               data[e_mh].round(2).astype(str).to_numpy(),
               data[r_mh].astype(str).to_numpy()]

listofcolerr1 = listofcol1
listofcolname1 = ['Star', lum, e_lum, r_lum, Teff, e_Teff, r_Teff, mh, e_mh, r_mh]
listofcolformat = ['{:s}']*10
listofcolerrformat = ['']*10

blistofcol = listofcol1
blistofcolerr = listofcolerr1
blistofcolname = listofcolname1
blistofcolformat = listofcolformat
blistofcolerrformat = listofcolerrformat

caption = 'Stellar parameters of the studied sample.'
filename = 'Tab.1_stellar_params.txt'
label = 'tab:stellar-params'
align = '@{\extracolsep{\\fill}}lrrrrrrrrrrrr'
footnote = 'References for the stellar parameters:'+';'.join([' {} {}'.format(key, refs_dict[key]) for key in refs_dict.keys()]) +\
'. Only the first 10 lines are shown. The full table can be accessed online.'
my_write_latex_table(blistofcol, blistofcolerr, blistofcolname, blistofcolformat, blistofcolerrformat, filename, 
	expandcolumns=True, scaption=caption, slabel=label, align=align, sfootnote=footnote)
# os.system(to_overleaf(filename, 'tables'))